In [1]:
!pip install --upgrade --quiet kuzu
## the extensions will install to ~/.kuzu/extension/0.10.0/win_amd64/xxx  refer https://extension.kuzudb.com/v0.10.0/win_amd64/xxx
import kuzu


# Create an empty on-disk database and connect to it
db = kuzu.Database("./demo_db_0.10")
print(db.get_version())
print(db.get_storage_version())
conn = kuzu.Connection(db)
# Create a graph with a node label and a property
try:
    conn.execute(
        """
     CREATE NODE TABLE Book (ID SERIAL, abstract STRING, title STRING, PRIMARY KEY (ID));
CREATE (b:Book {abstract: 'An exploration of quantum mechanics.', title: 'The Quantum World'});
CREATE (b:Book {abstract: 'A magic journey through time and space.', title: 'Chronicles of the Universe'});
CREATE (b:Book {abstract: 'An introduction to machine learning techniques.', title: 'Learning Machines'});
CREATE (b:Book {abstract: 'A deep dive into the history of ancient civilizations.', title: 'Echoes of the Past'});
CREATE (b:Book {abstract: 'A fantasy tale of dragons and magic.', title: 'The Dragons Call'});
        """
    )
except Exception as e:
    print(f"Error executing graph operations: {str(e)}")
# Execute Cypher query
conn.execute(
    """
LOAD FTS;
""");

response=conn.execute("""
CALL SHOW_INDEXES() RETURN *;
    """
)
while response.has_next():
    print(response.get_next())
response.close()
conn.close()
db.close()

0.10.0
38
Error executing graph operations: Binder exception: Book already exists in catalog.


In [ ]:
import kuzu


class KuzuSession:
    def __init__(self, db_path):
        self.db_path = db_path

    def __enter__(self):
        self.db = kuzu.Database(self.db_path)
        self.conn = kuzu.Connection(self.db)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
       # self.conn.close()
       # self.db.close()
        print("Session closed successfully.")

# Usage:
with KuzuSession("demo_db") as conn:
    result = conn.execute("MATCH (n) RETURN n LIMIT 1")
    print(result.get_as_df())

In [ ]:
from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_db") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 10")
    print(df)

RuntimeError: Error in Kùzu subprocess:
Traceback (most recent call last):
  File "e:\projects\kuzu-lite\util\safe_kuzu_subprocess.py", line 19, in kuzu_worker
    df = result.get_as_df()
         ^^^^^^^^^^^^^^^^
AttributeError: 'list' object has no attribute 'get_as_df'


In [6]:

from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_large") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 2000")
    print(df)

RuntimeError: Error in Kùzu subprocess:
Traceback (most recent call last):
  File "e:\projects\kuzu-lite\util\safe_kuzu_subprocess.py", line 9, in kuzu_worker
    db = kuzu.Database(db_path)
  File "d:\Tool\python\Lib\site-packages\kuzu\database.py", line 104, in __init__
    self.init_database()
    ~~~~~~~~~~~~~~~~~~^^
  File "d:\Tool\python\Lib\site-packages\kuzu\database.py", line 155, in init_database
    self._database = _kuzu.Database(  # type: ignore[union-attr]
                     ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        self.database_path,
        ^^^^^^^^^^^^^^^^^^^
    ...<6 lines>...
        self.checkpoint_threshold,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
RuntimeError: Runtime exception: Trying to read a database file with a different version. Database file version: 37, Current build storage version: 38
